Item 41 Consider Composing Functionality with Mix-in Classes

Things to Remember
- Avoid using multiple inheritance with instance attributes and __init__ if mix-in classes can achieve the same outcome.
- Use pluggable behaviors at the instance level to provide per-class customization when mix-in classes may require it.
- Mix-ins can include instance methods or class methods, depending on your needs.
- Compose mix-ins to create complex functionality from simple behaviors.


About \__dict\__
- A dictionary or other mapping object used to store an object's (writable) attributes. 
- Check the AttributeAccess file in the same folder for more details and examples   

About mix-ins
- A mix-in is a class that defines only a small set of additional methods for its child classes to provide.
- Mix-in classes don't define their own instance attributes nor require their __init__ constructor to be called.
- Mix-ins can include instance methods or class methods, depending on your needs.
- Compose mix-ins to create complex functionality from simple behaviors. 


In [31]:
# - you want to convert an object from its in-memory representation
#   to a dictionary that's ready for serialization
class ToDictMixin:
    def to_dict(self):
        # - this is why it's pluggable
        # - it will call the overridden version 
        #   if there is one as you use 'self' 
        #   to look for such implementation 
        return self._traverse_dict(self.__dict__)
    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            # - same here you look for the 
            #   overridden version first 
            output[key] = self._traverse(key, value)
        return output
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            # - since you are an instance of ToDictMixin
            #   you must have a to_dict method defined                
            #   so I will just use that
            return value.to_dict()
        if isinstance(value, dict):
            # - dict objects do not have __dict__
            # - we just traverse it as a dict   
            return self._traverse_dict(value)
        if isinstance(value, list): # return a list
            # - list objects do not have __dict__
            return [self._traverse(key, i) for i in value]
        if hasattr(value, '__dic__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value

In [32]:
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

In [33]:
tree = BinaryTree(10,
    left=BinaryTree(7, right=BinaryTree(9)),
    right=BinaryTree(13,left=BinaryTree(11)))
print(tree.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None}}, 'right': {'value': 13, 'left': {'value': 11, 'left': None, 'right': None}, 'right': None}}


In [34]:
# - make mix-ins generic functionality pluggable
#   so behaviors can be overriden when required
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None,
                 right=None, parent=None):
        super().__init__(value, left = left, right=right)
        self.parent = parent
    # - override _traverse (in ToDictMixin) to only prcocess
    #   values that matter 
    def _traverse(self, key, value):
        if (isinstance(value, BinaryTreeWithParent) and
                key == 'parent'):
            # - prevent cycles but not following
            #   the circular referencing properties (parent)     
            return value.value
        else:
            return super()._traverse(key, value)


In [35]:
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
print(root.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None, 'parent': 7}, 'parent': 10}, 'right': None, 'parent': None}


In [36]:
# - just to recap BinaryTreeWithParent 
#   is an instance of ToDictMixin 
assert (isinstance(root, ToDictMixin))

In [37]:
# - you also enable any class that has an
#   attribute of BinaryTreeWithParent
#   to automatically work with the 
#   ToDictMixin

class NamedSubTree(ToDictMixin):
    def __init__(self, name, tree_with_parent):
        self.name = name
        self.tree_with_parent = tree_with_parent

In [38]:
my_tree = NamedSubTree('foobar', root.left.right)
print(my_tree.to_dict()) # no infinite loop

{'name': 'foobar', 'tree_with_parent': {'value': 9, 'left': None, 'right': None, 'parent': 7}}


In [39]:
# compose mix-ins
import json

class JsonMixin:
    @classmethod
    def from_json(cls, data): # factory
        # - json.load: deserialize a 
        #   JSON document to a Python
        #   object
        kwargs = json.loads(data)
        # - **kwargs allows you to pass keyworded 
        #     variable length of arguments to 
        #     a function. 
        return cls(**kwargs)
    
    def to_json(self):
        return json.dumps(self.to_dict())  
   

In [40]:
# - you are designing a hierarchy of data classes
#   representing parts of a datacenter topology

class Switch(ToDictMixin, JsonMixin):
    def __init__(self, ports=None, speed=None):
        self.ports = ports
        self.speed = speed

class Machine(ToDictMixin, JsonMixin):
    def __init__(self, cores=None, ram=None, disk=None):
        self.cores = cores
        self.ram = ram
        self.disk = disk

class DatacenterRack(ToDictMixin, JsonMixin):
    def __init__(self, switch=None, machines=None):
        self.switch = Switch(**switch)
        self.machines = [
            Machine(**kwargs) for kwargs in machines]


In [42]:
serialized = """{
    "switch" : {"ports": 5, "speed": 1e9},
    "machines": [
        {"cores": 8, "ram": 32e9, "disk": 5e12},
        {"cores": 4, "ram": 16e9, "disk": 1e12},
        {"cores": 2, "ram": 4e9, "disk": 500e9}
    ]
}"""

deserialized = DatacenterRack.from_json(serialized)
roundtrip = deserialized.to_json()
assert(json.loads(serialized) == json.loads(roundtrip))